In [8]:
import numpy as np
from numba import njit
import torch

In [9]:
def generate_positions(dims=(100, 100), n=64):
    return np.random.uniform(0, dims, (n, 2))

In [10]:
positions = generate_positions(dims=(10, 10), n=5); positions

array([[1.12706016, 3.61816813],
       [8.08691906, 3.99137426],
       [9.28175179, 0.62590078],
       [8.44526255, 0.46204624],
       [2.28119671, 6.93673565]])

In [11]:
@njit
def distance_matrix_squared(pts):
    n = pts.shape[0]
    p = pts.shape[1]
    dist = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            s = 0
            for k in range(p):
                s += (pts[i, k] - pts[j, k])**2
            dist[i, j] = s
    return dist

In [12]:
# ignore "divide by zero" warning
def channel_gain_matrix(positions, wavelength):
    L_p0 = (wavelength / (4 * np.pi)) ** 2

    # TODO: add shadow and multipath
    H = L_p0 / distance_matrix_squared(positions)
    np.fill_diagonal(H, 0)
    return H

In [13]:
wavelengths = {
    "channel_37": 0.1249135, # m (2.400 GHz)
    "channel_38": 0.1235748, # m (2.426 GHz)
    "channel_39": 0.1208840, # m (2.480 GHz)
}
P_max = 10 # mW
noise = 0
shift_matrix_indicator = {
    "M" : 1,
    "nu": 1
}

In [14]:
H = channel_gain_matrix(positions, wavelengths["channel_37"]); H

<ipython-input-12-e8ec87e6f9f0>:6: RuntimeWarning: divide by zero encountered in true_divide
  H = L_p0 / distance_matrix_squared(positions)


array([[0.00000000e+00, 2.03400109e-06, 1.30955720e-06, 1.55563512e-06,
        8.00406653e-06],
       [2.03400109e-06, 0.00000000e+00, 7.74731747e-06, 7.85164712e-06,
        2.33142811e-06],
       [1.30955720e-06, 7.74731747e-06, 0.00000000e+00, 1.35995961e-04,
        1.11228944e-06],
       [1.55563512e-06, 7.85164712e-06, 1.35995961e-04, 0.00000000e+00,
        1.23639764e-06],
       [8.00406653e-06, 2.33142811e-06, 1.11228944e-06, 1.23639764e-06,
        0.00000000e+00]])

In [15]:
tx_powers = np.random.uniform(0, P_max, len(positions)); tx_powers # in mW

array([7.51518882, 1.53526905, 5.53704228, 2.70820626, 7.05643357])

In [16]:
def calc_rssi_map(H, tx_powers):
    return H * tx_powers

In [17]:
rssi_map = calc_rssi_map(H, tx_powers); rssi_map

array([[0.00000000e+00, 3.12273893e-06, 7.25107361e-06, 4.21298077e-06,
        5.64801637e-05],
       [1.52859023e-05, 0.00000000e+00, 4.28972244e-05, 2.12638799e-05,
        1.64515676e-05],
       [9.84156966e-06, 1.18942168e-05, 0.00000000e+00, 3.68305115e-04,
        7.84879651e-06],
       [1.16908916e-05, 1.20543908e-05, 7.53015389e-04, 0.00000000e+00,
        8.72455777e-06],
       [6.01520713e-05, 3.57936943e-06, 6.15879364e-06, 3.34841982e-06,
        0.00000000e+00]])

In [18]:
@njit
def calc_sinr_matrix(rssi_map):
    n = rssi_map.shape[0]
    sinr_matrix = np.empty(rssi_map.shape)
    totals = np.sum(rssi_map, axis=1) + noise
    for i in range(n):
        sinr_matrix[i, :] = rssi_map[i, :] / (totals[i] - rssi_map[i, :])
    return sinr_matrix

In [19]:
sinr_matrix = calc_sinr_matrix(rssi_map); sinr_matrix

array([[0.00000000e+00, 4.59603336e-02, 1.13624904e-01, 6.30176544e-02,
        3.87200686e+00],
       [1.89621580e-01, 0.00000000e+00, 8.09360981e-01, 2.84906103e-01,
        2.07075991e-01],
       [2.53617244e-02, 3.08143939e-02, 0.00000000e+00, 1.24492245e+01,
        2.01230089e-02],
       [1.51085257e-02, 1.55856093e-02, 2.31912255e+01, 0.00000000e+00,
        1.12319767e-02],
       [4.59646890e+00, 5.13839533e-02, 9.18128570e-02, 4.79096951e-02,
        0.00000000e+00]])

In [20]:
def calc_shannon_capacity_matrix(sinr_matrix):
    return np.log2(1 + sinr_matrix)

In [21]:
shannon_capacity_matrix = calc_shannon_capacity_matrix(sinr_matrix); shannon_capacity_matrix

array([[0.        , 0.06482814, 0.15526338, 0.08816556, 2.28451616],
       [0.25050272, 0.        , 0.85548026, 0.36166294, 0.2715165 ],
       [0.03613295, 0.04378459, 0.        , 3.74945108, 0.02874313],
       [0.02163397, 0.02231186, 4.59641195, 0.        , 0.01611399],
       [2.48451684, 0.07228962, 0.12672559, 0.0675144 , 0.        ]])

In [23]:
def calc_shift_matrix(H, noise=1):
    n = H.shape[0]

    # shifted diagonal
    diag = np.array([H[i, (i+1) % n] for i in range(n)])
    indicator = np.empty(H.shape)
    for i in range(len(H)):
        indicator[i, :] = np.minimum(H[i, (i+1) % n], diag)

    nu = shift_matrix_indicator["nu"]
    M = shift_matrix_indicator["M"]

    constant = M * ((P_max / noise) ** (nu - 1))
    indicator = constant * (indicator ** nu)

    indicator = H >= indicator
    return np.multiply(H, indicator)

In [24]:
shift_matrix = calc_shift_matrix(H); shift_matrix

array([[0.00000000e+00, 2.03400109e-06, 0.00000000e+00, 1.55563512e-06,
        8.00406653e-06],
       [2.03400109e-06, 0.00000000e+00, 7.74731747e-06, 7.85164712e-06,
        0.00000000e+00],
       [0.00000000e+00, 7.74731747e-06, 0.00000000e+00, 1.35995961e-04,
        0.00000000e+00],
       [1.55563512e-06, 7.85164712e-06, 1.35995961e-04, 0.00000000e+00,
        1.23639764e-06],
       [8.00406653e-06, 0.00000000e+00, 0.00000000e+00, 1.23639764e-06,
        0.00000000e+00]])

In [25]:
def z(alpha, S, y):
    S_ = S
    zz = alpha[0] * np.dot(S_, y)
    for k in range(1, len(alpha)):
        S_ = np.dot(S_, S)
        zz += alpha[k] * np.dot(S_, y)
    return zz

In [26]:
z_l = z([0.1, 0.2], shift_matrix, tx_powers); z_l

array([6.38196670e-06, 7.94654919e-06, 3.80414208e-05, 7.85590243e-05,
       6.35034550e-06])

In [27]:
def ReLU(z_l):
    return np.maximum(0, z_l)

In [28]:
y_l = ReLU(z_l); y_l

array([6.38196670e-06, 7.94654919e-06, 3.80414208e-05, 7.85590243e-05,
       6.35034550e-06])